# Script 01: Pre-processing

The overall goal of the project is the prediction of a relapse of porn-addicted study participants.

The data we have:

c - Demographic information.

q - A questionnaire filled out from time to time. Domain expert tell us that the results should be valid for up to 14 days.

r - A questionnaire (reminder) to be filled out up to three times a day. Here, participants note if they relapsed and report their current feelings.

This script pre-processes our data. In the end, we have our final dataset. Each row is an instance of a reminder r, for which we have valid info from q and c. We will then try to predict the relapse by q, c, and the answers given in the previous r.

Please note: For privacy reasons, we cannot show the contents of the data.

In [1]:
# window extension & maximaze rows and columns & imports

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

#pd.set_option("display.max_rows", 100, "display.max_columns", 100)

In [2]:
# read in the data
c = pd.read_csv("data/characteristics.csv")
q = pd.read_csv("data/screen.csv", na_values = 'NA')
r = pd.read_csv("data/reminder.csv", parse_dates=['activityDate'])

# Prepare dataset q

In [3]:
#q.head()

In [4]:
# keep only relevant columns from q
q = q[['responderId','activityDate','screen_use_guilt', 'screen_self_satysfaction','screen_tried_to_stop','screen_use_to_cope','screen_hard_to_resist','screen_use_more_last_week','screen_proud',
       'main_profile', 'gender', 'age', 'city','treatment']]

In [5]:
# looking into gender distribution - we are only going to keep the "MALE" entries, as we do not have enough data for the other options
q['gender'].value_counts()

MALE       13296
FEMALE      3799
unknown      513
Name: gender, dtype: int64

In [6]:
# keep only "watch_porn" and "male"
q = q[(q.gender == "MALE") & (q.main_profile == "watch_porn")]
q.responderId.nunique()

668

In [7]:
# calculate screen_gen variable and add it to our df
q.loc[:, 'screen_gen'] = q[['screen_hard_to_resist', 'screen_tried_to_stop', 'screen_use_guilt', 'screen_use_more_last_week', 'screen_use_to_cope']].mean(axis='columns')

In [8]:
#q.head()

# Prepare dataset r

In [9]:
#r.head()

In [10]:
# set relapse as class 1, no relapse as class 0
r['relapse'].replace({0:1, 1:0}, inplace=True)

In [11]:
# keep only "watch_porn" in main_profile and "male" in r
r = r[(r.gender == "MALE") & (r.main_profile == "watch_porn")]

# keep only relevant columns from r
r = r.drop(['age', 'city', 'treatment', 'main_profile', 'gender'], axis=1)

# changing name of a column becuase q and r have the same variable 'activityDate' that contain different data 
r = r.rename(columns={"activityDate": "this_date"})

In [12]:
#r.head()

# Prepare dataset c

In [13]:
#c.head()

In [14]:
# keep only "watch_porn" in main_profile and "male" in c
c = c[(c.gender == "MALE") & (c.main_profile == "watch_porn")]

# keep only relevant columns from r
c = c.drop(['age', 'city', 'treatment', 'main_profile', 'gender', 'watch_porn'], axis=1)

In [15]:
#c.head()

# Merge q and c

In [16]:
# add prefixes so we immediatly see which column is from q and which from r
c = c.add_prefix('c_')
c = c.rename(columns={"c_responderId": "responderId"})
q = q.add_prefix('q_')
q = q.rename(columns={"q_responderId": "responderId"})
r = r.add_prefix('r_')
r = r.rename(columns={"r_responderId": "responderId"})

# add identifying column
q['type'] = 'q'
r['type'] = 'r'

In [17]:
# merge the two dataframes
cq = pd.merge(q, c, on=["responderId"])

In [18]:
#cq.head()

# qc: Calculate time difference between two consecutive q questionnaires

In [19]:
# set datetime format
cq['q_activityDate'] = pd.to_datetime(cq['q_activityDate']).dt.tz_localize(None)

# sort q_activityDate
cq = cq.sort_values(['responderId', 'q_activityDate'], ascending = (True, True))

# calculate delta to previous q
cq['q_delta'] = (cq['q_activityDate']-cq['q_activityDate'].shift()).fillna(pd.Timedelta(seconds=0))

In [20]:
# set first q to 0 seconds time delta
# https://stackoverflow.com/questions/46242488/change-first-element-of-each-group-in-pandas-dataframe
cq.loc[cq.groupby('responderId')['q_delta'].head(1).index, 'q_delta'] = pd.Timedelta("0 seconds")

# r: Calculate time difference between two consecutive r questionnaires

In [21]:
# set datetime format
r['r_this_date'] = pd.to_datetime(r['r_this_date']).dt.tz_localize(None)

# sort r_this_date
r = r.sort_values(['responderId', 'r_this_date'], ascending = (True, True))

In [22]:
# group-wise (by participantId): shift all values except responderId, date, q_age, q_city, q_treatment, r_relapse to the next row

# https://stackoverflow.com/questions/26280345/pandas-shift-down-values-by-one-row-within-a-group/31212472

columns_list_to_shift = ['r_presser',
 'r_anger',
 'r_uncertain',
 'r_uncertain_scale',
 'r_arousal',
 'r_mood_valence',
 'r_anxiety',
 'r_procrastination',
 'r_craving',
 'r_hunger',
 'r_lonlyness',
 'r_fatigue',
 'r_addiction_succumb_times',
 'r_addiction_succumb_consequences',
 'r_persevere_difficulty',
 'r_addiction_succumb_reason_0',
 'r_addiction_succumb_reason_1',
 'r_addiction_succumb_reason_2',
 'r_addiction_succumb_reason_3',
 'r_addiction_succumb_reason_4',
 'r_addiction_succumb_reason_5',
 'r_addiction_succumb_reason_6',
 'r_addiction_succumb_reason_7',
 'r_addiction_succumb_reason_8',
 'r_addiction_succumb_reason_9',
 'r_addiction_succumb_reason_10',
 'r_addiction_succumb_reason_11',
 'r_addiction_succumb_reason_12',
 'r_persevere_motivation_0',
 'r_persevere_motivation_1',
 'r_persevere_motivation_2',
 'r_persevere_motivation_3',
 'r_persevere_motivation_4',
 'r_persevere_motivation_5',
 'r_persevere_motivation_6',
 'r_persevere_motivation_7',
 'r_persevere_motivation_8',
 'r_persevere_motivation_9',
 'r_addiction_succumb_consequences2_0',
 'r_addiction_succumb_consequences2_1',
 'r_addiction_succumb_consequences2_2',
 'r_addiction_succumb_consequences2_3',
 'r_addiction_succumb_consequences2_4',
 'r_addiction_succumb_consequences2_5',
 'r_addiction_succumb_consequences2_6',
 'r_addiction_succumb_consequences2_7',
 'r_addiction_succumb_consequences2_8',
 'r_addiction_succumb_consequences2_9',
 'r_addiction_succumb_consequences2_10',
 'r_addiction_succumb_consequences2_11',
 'r_addiction_succumb_consequences2_12',
 'r_addiction_succumb_consequences2_13',
 'r_addiction_succumb_consequences2_14',
 'r_addiction_succumb_consequences2_15',
 'r_addiction_succumb_consequences2_16',
 'r_addiction_succumb_consequences2_18',
 'r_addiction_succumb_consequences2_19',
 'r_addiction_succumb_consequences2_20',
 'r_addiction_succumb_consequences2_21',
 'r_addiction_succumb_consequences2_22']

for col in columns_list_to_shift:
    col_shifted = col+'_shifted_1'
    r[col_shifted] = r.groupby(['responderId'])[col].shift(1)

In [23]:
# delete all rows whose time diff to previous row is greater than 24h - step 1: write down time-diff
# https://stackoverflow.com/questions/16777570/calculate-time-difference-between-pandas-dataframe-indices
r['r_delta'] = (r['r_this_date']-r['r_this_date'].shift()).fillna(pd.Timedelta(seconds=0))

# to have consistent data, we need to delete the first row of each group
# https://stackoverflow.com/questions/31226142/python-pandas-delete-the-first-row-by-group
r = r.groupby('responderId').apply(lambda group: group.iloc[1:, 1:])

In [24]:
# clean-up
r = r.reset_index()
r = r.drop(columns=columns_list_to_shift)
r = r.drop(columns=['level_1'])

In [25]:
#r.head()

# Concat qc and r

In [26]:
cqr = pd.concat([cq, r])

In [27]:
#cqr.head()

# Define a single "date" column

In [28]:
# add date column
cqr['date'] = pd.NaT

In [29]:
# reorder columns - put ID and dates to the left for readability

cqr = cqr[['responderId',
 'date',
 'r_this_date',
 'q_activityDate',
 'type',
 'r_delta',
 'q_delta',
 'q_screen_use_guilt',
 'q_screen_self_satysfaction',
 'q_screen_tried_to_stop',
 'q_screen_use_to_cope',
 'q_screen_hard_to_resist',
 'q_screen_use_more_last_week',
 'q_screen_proud',
 'q_main_profile',
 'q_gender',
 'q_age',
 'q_city',
 'q_treatment',
 'q_screen_gen',
 'c_addiction_default',
 'c_alco',
 'c_amphetamine',
 'c_caffeine',
 'c_cocaine',
 'c_compulsive_sex',
 'c_compulsive_shopping',
 'c_eating',
 'c_gambling',
 'c_games',
 'c_hallucinogen',
 'c_heroine',
 'c_marijuana',
 'c_masturbation',
 'c_nicotine',
 'c_painkillers',
 'c_series',
 'c_stimulants',
 'c_sweets',
 'c_work',
 'r_relapse',
 'r_addiction_succumb_reason_other',
 'r_persevere_motivation_other',
 'r_presser_shifted_1',
 'r_anger_shifted_1',
 'r_uncertain_shifted_1',
 'r_uncertain_scale_shifted_1',
 'r_arousal_shifted_1',
 'r_mood_valence_shifted_1',
 'r_anxiety_shifted_1',
 'r_procrastination_shifted_1',
 'r_craving_shifted_1',
 'r_hunger_shifted_1',
 'r_lonlyness_shifted_1',
 'r_fatigue_shifted_1',
 'r_addiction_succumb_times_shifted_1',
 'r_addiction_succumb_consequences_shifted_1',
 'r_persevere_difficulty_shifted_1',
 'r_addiction_succumb_reason_0_shifted_1',
 'r_addiction_succumb_reason_1_shifted_1',
 'r_addiction_succumb_reason_2_shifted_1',
 'r_addiction_succumb_reason_3_shifted_1',
 'r_addiction_succumb_reason_4_shifted_1',
 'r_addiction_succumb_reason_5_shifted_1',
 'r_addiction_succumb_reason_6_shifted_1',
 'r_addiction_succumb_reason_7_shifted_1',
 'r_addiction_succumb_reason_8_shifted_1',
 'r_addiction_succumb_reason_9_shifted_1',
 'r_addiction_succumb_reason_10_shifted_1',
 'r_addiction_succumb_reason_11_shifted_1',
 'r_addiction_succumb_reason_12_shifted_1',
 'r_persevere_motivation_0_shifted_1',
 'r_persevere_motivation_1_shifted_1',
 'r_persevere_motivation_2_shifted_1',
 'r_persevere_motivation_3_shifted_1',
 'r_persevere_motivation_4_shifted_1',
 'r_persevere_motivation_5_shifted_1',
 'r_persevere_motivation_6_shifted_1',
 'r_persevere_motivation_7_shifted_1',
 'r_persevere_motivation_8_shifted_1',
 'r_persevere_motivation_9_shifted_1',
 'r_addiction_succumb_consequences2_0_shifted_1',
 'r_addiction_succumb_consequences2_1_shifted_1',
 'r_addiction_succumb_consequences2_2_shifted_1',
 'r_addiction_succumb_consequences2_3_shifted_1',
 'r_addiction_succumb_consequences2_4_shifted_1',
 'r_addiction_succumb_consequences2_5_shifted_1',
 'r_addiction_succumb_consequences2_6_shifted_1',
 'r_addiction_succumb_consequences2_7_shifted_1',
 'r_addiction_succumb_consequences2_8_shifted_1',
 'r_addiction_succumb_consequences2_9_shifted_1',
 'r_addiction_succumb_consequences2_10_shifted_1',
 'r_addiction_succumb_consequences2_11_shifted_1',
 'r_addiction_succumb_consequences2_12_shifted_1',
 'r_addiction_succumb_consequences2_13_shifted_1',
 'r_addiction_succumb_consequences2_14_shifted_1',
 'r_addiction_succumb_consequences2_15_shifted_1',
 'r_addiction_succumb_consequences2_16_shifted_1',
 'r_addiction_succumb_consequences2_18_shifted_1',
 'r_addiction_succumb_consequences2_19_shifted_1',
 'r_addiction_succumb_consequences2_20_shifted_1',
 'r_addiction_succumb_consequences2_21_shifted_1',
 'r_addiction_succumb_consequences2_22_shifted_1']]

In [30]:
# set date column
# https://stackoverflow.com/questions/58303311/replace-a-column-with-another-column-if-another-is-not-null-in-pandas-dataframe
cqr['date'] = np.where(~cqr['r_this_date'].isnull(), cqr['r_this_date'], cqr['q_activityDate'])

In [31]:
# drop date columns that are now superfluous
cqr = cqr.drop(columns=['r_this_date', 'q_activityDate', 'q_main_profile', 'q_gender'])

# Fill missing constant values from c

In [32]:
# q_age, q_city, q_treatment, and the c columns are constant for each responderId; propagate to rows where it is NaN

columns_to_fill = ['q_age',
 'q_city',
 'q_treatment',
 'c_addiction_default',
 'c_alco',
 'c_amphetamine',
 'c_caffeine',
 'c_cocaine',
 'c_compulsive_sex',
 'c_compulsive_shopping',
 'c_eating',
 'c_gambling',
 'c_games',
 'c_hallucinogen',
 'c_heroine',
 'c_marijuana',
 'c_masturbation',
 'c_nicotine',
 'c_painkillers',
 'c_series',
 'c_stimulants',
 'c_sweets',
 'c_work']

for col in columns_to_fill:
    df_tmp = cqr.groupby(['responderId'])[col].first()
    cqr[col] = np.where(~cqr[col].isnull(), cqr[col], df_tmp[cqr['responderId']])

In [33]:
# sort by responderId and date
cqr = cqr.sort_values(['responderId', 'date'], ascending = (True, True))

In [34]:
# cqr.head()

# Forward fill q values for up to 14 days
The domain experts tell us that the answers to the q questionnaire are valid for up to 14 days.

We forward fill the answers for up to 14 days and will remove the rows where we do not have q info.

In [35]:
%%time

# loop over all responderIds and create new DFs that contain the ffill values of q for max 14 days

list_of_responderIds = cqr['responderId'].unique()

result_dfs = []

cnt = 0
for responderId in list_of_responderIds:
    cnt += 1
    #print('processing user '+str(cnt)+' of '+str(len(list_of_responderIds))+': '+str(responderId))

    # set datetime index; then resample to minutes
    df_tmp = cqr.loc[cqr['responderId'] == responderId].copy()
    #display(df_tmp)
    df_tmp['date'] = pd.to_datetime(df_tmp['date'], format='%Y')
    # if two timestamps are the same: increase the second one one minute
    # if previous date is the same: increase current date
    df_tmp.loc[df_tmp['date'] == df_tmp['date'].shift(-1), 'date'] = df_tmp['date'] + pd.Timedelta(minutes=1)
    df_tmp = df_tmp.set_index(['date'])
    df_tmp = df_tmp.resample('1T').max() # this takes the max value; should not matter, as we are expanding the df, not shortening
    #print(df_tmp)

    # ffill (copy forward) the q results for up to 14 days
    # 14 days is 20160 minutes
    cols = ['q_screen_use_guilt', 'q_screen_self_satysfaction', 'q_screen_tried_to_stop', 'q_screen_use_to_cope', 'q_screen_hard_to_resist', 'q_screen_use_more_last_week', 'q_screen_proud', 'q_screen_gen']
    df_tmp.loc[:,cols] = df_tmp.loc[:,cols].fillna(method="ffill", limit=20160)
    
    # drop all unimportant rows (identified by type = NaN)
    df_tmp = df_tmp[df_tmp['type'].notna()]
    #display(df_tmp)
    df_tmp = df_tmp.reset_index()

    result_dfs.append(df_tmp)

# concat all results
df = pd.concat(result_dfs)

Wall time: 2h 9min 14s


# Clean-up
We remove rows we cannot use; columns we we cannot use; and rows with missing values that we should not impute.

In [36]:
# remove type q rows - info is copied to r rows - this includes deletion of those that never filled our r
df = df[df['type'] != 'q']

# delete those with missing relapse info
df = df[df['r_relapse'].notna()]

# delete all rows whose time diff to previous row is greater than 24h - step 2: delete the rows
df = df.reset_index() # so we do not delete multiple rows with one index
df = df.drop(df[df['r_delta'].dt.days > 0].index)
df = df.drop(columns=['index'])

In [37]:
# remove unnecessary columns
df = df.drop(columns=['q_delta', 'type', 'c_addiction_default', 'r_persevere_motivation_other', 'r_addiction_succumb_reason_other'])

In [38]:
# remove rows where q or c is not available
clmns = ['q_screen_use_guilt',
 'q_screen_self_satysfaction',
 'q_screen_tried_to_stop',
 'q_screen_use_to_cope',
 'q_screen_hard_to_resist',
 'q_screen_use_more_last_week',
 'q_screen_proud',
 'q_age',
 'q_city',
 'q_treatment',
 'q_screen_gen',
 'c_alco',
 'c_amphetamine',
 'c_caffeine',
 'c_cocaine',
 'c_compulsive_sex',
 'c_compulsive_shopping',
 'c_eating',
 'c_gambling',
 'c_games',
 'c_hallucinogen',
 'c_heroine',
 'c_marijuana',
 'c_masturbation',
 'c_nicotine',
 'c_painkillers',
 'c_series',
 'c_stimulants',
 'c_sweets',
 'c_work']

for cl in clmns:
    df = df[df[cl].notna()]

# Create new features: day of the week, time of day, time since last reminder r

In [39]:
# day of the week
df['day_of_week'] = df['date'].dt.weekday

In [40]:
# time of day
# from date -> take time -> map to double/float
df['time_of_day'] = df['date'].dt.time
df['time_of_day_minutes'] = (pd.to_timedelta(df['time_of_day'].astype(str)) / pd.offsets.Minute(1))
# minutes in a day: 1440
df['time_of_day'] = df['time_of_day_minutes'] / 1440.0
df = df.drop(columns = ['time_of_day_minutes'])

In [41]:
# time since last reminder
df['r_delta_f'] = (df['r_delta'] / np.timedelta64(1,'s') / 3600).astype(float)

In [42]:
# drop columsn we do not need anymore
df = df.drop(columns=['date', 'r_delta'])

# Imputing missing values
We impute missing values with the help of our domain experts.

In [43]:
#df.info()

In [44]:
# we can impute NaN with 0 for these columns

clmns = ['r_addiction_succumb_reason_0_shifted_1',
 'r_addiction_succumb_reason_1_shifted_1',
 'r_addiction_succumb_reason_2_shifted_1',
 'r_addiction_succumb_reason_3_shifted_1',
 'r_addiction_succumb_reason_4_shifted_1',
 'r_addiction_succumb_reason_5_shifted_1',
 'r_addiction_succumb_reason_6_shifted_1',
 'r_addiction_succumb_reason_7_shifted_1',
 'r_addiction_succumb_reason_8_shifted_1',
 'r_addiction_succumb_reason_9_shifted_1',
 'r_addiction_succumb_reason_10_shifted_1',
 'r_addiction_succumb_reason_11_shifted_1',
 'r_addiction_succumb_reason_12_shifted_1',
 'r_persevere_motivation_0_shifted_1',
 'r_persevere_motivation_1_shifted_1',
 'r_persevere_motivation_2_shifted_1',
 'r_persevere_motivation_3_shifted_1',
 'r_persevere_motivation_4_shifted_1',
 'r_persevere_motivation_5_shifted_1',
 'r_persevere_motivation_6_shifted_1',
 'r_persevere_motivation_7_shifted_1',
 'r_persevere_motivation_8_shifted_1',
 'r_persevere_motivation_9_shifted_1',
 'r_addiction_succumb_consequences2_0_shifted_1',
 'r_addiction_succumb_consequences2_1_shifted_1',
 'r_addiction_succumb_consequences2_2_shifted_1',
 'r_addiction_succumb_consequences2_3_shifted_1',
 'r_addiction_succumb_consequences2_4_shifted_1',
 'r_addiction_succumb_consequences2_5_shifted_1',
 'r_addiction_succumb_consequences2_6_shifted_1',
 'r_addiction_succumb_consequences2_7_shifted_1',
 'r_addiction_succumb_consequences2_8_shifted_1',
 'r_addiction_succumb_consequences2_9_shifted_1',
 'r_addiction_succumb_consequences2_10_shifted_1',
 'r_addiction_succumb_consequences2_11_shifted_1',
 'r_addiction_succumb_consequences2_12_shifted_1',
 'r_addiction_succumb_consequences2_13_shifted_1',
 'r_addiction_succumb_consequences2_14_shifted_1',
 'r_addiction_succumb_consequences2_15_shifted_1',
 'r_addiction_succumb_consequences2_16_shifted_1',
 'r_addiction_succumb_consequences2_18_shifted_1',
 'r_addiction_succumb_consequences2_19_shifted_1',
 'r_addiction_succumb_consequences2_20_shifted_1',
 'r_addiction_succumb_consequences2_21_shifted_1',
 'r_addiction_succumb_consequences2_22_shifted_1']

for cl in clmns:
    df[cl] = df[cl].fillna(0)

In [45]:
# columns to impute:

# r_addiction_succumb_times_shifted_1 - NaN should just be 0 (meaning 0 relapses)
df['r_addiction_succumb_times_shifted_1'] = df['r_addiction_succumb_times_shifted_1'].fillna(0)

# r_addiction_succumb_consequences_shifted_1 - scale, on which relapsed users rated how strong the consequences were. Impute 0 for non-relpased users
df['r_addiction_succumb_consequences_shifted_1'] = df['r_addiction_succumb_consequences_shifted_1'].fillna(0)

# r_persevere_difficulty_shifted_1 - scale, on which non-relapsed users rated how difficult the abstinence was. Impute 10 for relapsed users
df['r_persevere_difficulty_shifted_1'] = df['r_persevere_difficulty_shifted_1'].fillna(10)

# r_uncertain_shifted_1 - "do you feel there is some uncertainy in your life?" 0 means "yes"; 1 and 2 mean "no" or "don't know"
# r_uncertain_scale_shifted_1 - only answered if r_uncertain_shifted_1 was answered with yes/1; scale how strong the uncertainy is - impute 0 for users answering 1 or 2 to 'r_uncertain_shifted_1'
df['r_uncertain_scale_shifted_1'] = df['r_uncertain_scale_shifted_1'].fillna(0)

In [46]:
# no missing values anymore
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11645 entries, 0 to 16444
Data columns (total 95 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   responderId                                     11645 non-null  float64
 1   q_screen_use_guilt                              11645 non-null  float64
 2   q_screen_self_satysfaction                      11645 non-null  float64
 3   q_screen_tried_to_stop                          11645 non-null  float64
 4   q_screen_use_to_cope                            11645 non-null  float64
 5   q_screen_hard_to_resist                         11645 non-null  float64
 6   q_screen_use_more_last_week                     11645 non-null  float64
 7   q_screen_proud                                  11645 non-null  float64
 8   q_age                                           11645 non-null  object 
 9   q_city                                 

# Store final dataset

In [47]:
df.to_csv('data/final_dataset.csv', index=False)